# On-the-fly ggame computation

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import streamlit as st
import sklearn.metrics.pairwise as sklpw
from PIL import Image
from fuzzywuzzy import fuzz
import re

In [2]:
# LOAD DATA
allgamedata_df = pd.read_pickle('datasources/BGG_FINAL.pkl') # USE ONLY FOR URLS
allgamedocvects = np.load('datasources/allgamedocvects.npz')['arr_0']
finalgamelist_df = pd.read_pickle('datasources/BGG_GameSimilarityKeyP4.pkl')
finalgamelist_df.reset_index(drop=True,inplace=True) # So that row ids are indices to gamevector array

# FOr gameplay-based vectors
bgg_gameplay_df = pd.read_pickle('datasources/bgg_gameplayfeatures.pkl')
bgg_gameplay_df.dropna(inplace=True) # Some not-NAs here...
bgg_gameplay_df.reset_index(drop=True,inplace=True)
allgamePLAYdocvects = np.array(bgg_gameplay_df.iloc[:,1:]) # Create right here

## Verify data sizes

In [3]:
print('finalgamelist_df {} should match allgamedocvects {}'.format(finalgamelist_df.shape,allgamedocvects.shape))

finalgamelist_df (18561, 3) should match allgamedocvects (18561, 100)


In [4]:
print('bgg_gameplay_df {} should match allgamePLAYdocvects {}'.format(bgg_gameplay_df.shape,allgamePLAYdocvects.shape))

bgg_gameplay_df (18500, 272) should match allgamePLAYdocvects (18500, 271)


In [10]:
# FUNCTIONS

def getcompute_similar_by_gameplay(gamename,allgamedata_df, bgg_gameplay_df, allgamePLAYdocvects):
    # Get game rank from game name, and matrix index from rank
  
    gamerank = list(allgamedata_df.loc[allgamedata_df['game_name']==gamename,'game_rank'])[0]
    gamerank = int(gamerank)
    gamerank_idx = list(bgg_gameplay_df.index[bgg_gameplay_df['game_rank']==gamerank])[0]
    #print(gamename, gamerank_idx)
    
    mygamePLAYvector = allgamePLAYdocvects[gamerank_idx,:] 
    mygamePLAYvector = mygamePLAYvector.reshape(-1,1)

    mysimilarities_gp = []
    for t in range(0,allgamePLAYdocvects.shape[0]):
        currgamevect_gp = allgamePLAYdocvects[t,:]
        currgamevect_gp = currgamevect_gp.reshape(-1,1)
        dum = sklpw.cosine_similarity(currgamevect_gp.T,mygamePLAYvector.T)
        mysimilarities_gp.append(dum[0][0])
    mycompleteGPsimlist_df = pd.concat((pd.DataFrame({'game_rank':bgg_gameplay_df['game_rank']}),pd.DataFrame({'GameplaySimilarity':mysimilarities_gp})),axis=1)
    return mycompleteGPsimlist_df


def getcompute_similar_games_by_id(mygameid,mygamename,allgamedata_df,allgamedocvects,finalgamelist_df,bgg_gameplay_df, allgamePLAYdocvects,W1,W2):
    myvectid = mygameid
    mygamevector = allgamedocvects[myvectid,:]
    mygamevector= mygamevector.reshape(-1,1)
    mysimilarities = []
    for t in range(0,allgamedocvects.shape[0]):
        currgamevect = allgamedocvects[t,:]
        currgamevect = currgamevect.reshape(-1,1)
        dum = sklpw.cosine_similarity(currgamevect.T,mygamevector.T)
        mysimilarities.append(dum[0][0])
    mycompletesimlist_df = pd.concat((finalgamelist_df['game_rank'],finalgamelist_df['game_name'],pd.DataFrame({'Similarity':mysimilarities})),axis=1)
    # Get this also for GAMEPLAY data
    mycompleteGPsimlist_df = getcompute_similar_by_gameplay(mygamename,allgamedata_df, bgg_gameplay_df, allgamePLAYdocvects)

    # PUt sim lists together
    mycompletesimlist_df  = mycompletesimlist_df.astype({'game_rank':'int32'},copy=True)
    mycompleteGPsimlist_df = mycompleteGPsimlist_df.astype({'game_rank':'int32'},copy=True)
    mycompletesimlist_df.set_index('game_rank',inplace=True)
    mycompleteGPsimlist_df.set_index('game_rank',inplace=True)
    # Do it
    myFINALsimlist_df = mycompletesimlist_df.join(mycompleteGPsimlist_df,how='inner')
    
    
    weightedsimilarity = (np.array(myFINALsimlist_df['Similarity'])*W1) + (np.array(myFINALsimlist_df['GameplaySimilarity'])*W2)
    myFINALsimlist_df['WghtdSimilarity'] = weightedsimilarity
    
    
    myFINALsimlist_df.sort_values(by='WghtdSimilarity',ascending=False,inplace=True)
    mytop10simlist_df = myFINALsimlist_df[1:11]
    # Create output list
    urllist=[]
    for gamename in mytop10simlist_df['game_name']:
        urllist.append(list(allgamedata_df.loc[allgamedata_df['game_name']==gamename,'bgg_url'])[0])
    #mytop10simlist_df = pd.DataFrame({'Game':mytop10simlist_df['game_name'],'Similarity':mytop10simlist_df['WghtdSimilarity'],'url':urllist})
    mytop10simlist_df = pd.DataFrame({'Game':mytop10simlist_df['game_name'],'url':urllist})
    mytop10simlist_df.reset_index(drop=True,inplace=True)
    mytop10simlist_df.index = mytop10simlist_df.index+1
    return mytop10simlist_df
    
    
    
    #mycompletesimlist_df.sort_values(by='Similarity',ascending=False,inplace=True)
    #mytop10simlist_df = mycompletesimlist_df[1:11]
    # Create output list
    #urllist=[]
    #for gamename in mytop10simlist_df['gamename']:
    #    urllist.append(list(allgamedata_df.loc[allgamedata_df['game_name']==gamename,'bgg_url'])[0])
    #mytop10simlist_df = pd.DataFrame({'Game':mytop10simlist_df['gamename'],'Similarity':mytop10simlist_df['Similarity'],'url':urllist})
    #mytop10simlist_df.reset_index(drop=True,inplace=True)
    #mytop10simlist_df.index = mytop10simlist_df.index+1
    #return mytop10simlist_df


def get_real_name_fuzzy(usergamename,finalgamelist_df):
    # Clean up
    usergamename = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", usergamename)
    #usergamename = re.sub(r"\d+", "", usergamename) # Maybe dont remove numbers?
    gamename_matchlist = [fuzz.token_sort_ratio(x,usergamename) for x in finalgamelist_df['game_name']]
    possiblegame_idx  = [i for i, x in enumerate(gamename_matchlist) if x == max(gamename_matchlist)]
    possiblegame_idx = possiblegame_idx[0] # Get first, make it number
    #possiblegame_name = list(finalgamelist_df.loc[finalgamelist_df['idx']==possiblegame_idx,'gamename'])[0]
    possiblegame_name = finalgamelist_df.iloc[possiblegame_idx,2]
    #print('Best match: {}'.format(possiblegame_name))
    
    return possiblegame_name,max(gamename_matchlist)

def make_clickable(url,text): # Make liks in pd df for url in table
    return f'<a target="_blank" href="{url}">{text}</a>'

def streamlitify_df(df):
    # Get original URLS
    df['Game_link'] = [make_clickable(a,b) for a,b in zip(list(df['url']),list(df['Game']))]
    return df


In [6]:
# CREATES THE DEMO GAME LIST
allgamedata_df['numeric_ranks']=[int(x) for x in allgamedata_df['game_rank']]
topranked_df = pd.DataFrame(allgamedata_df.loc[allgamedata_df['numeric_ranks']<=50,'game_name']) # To go back n forth
topranked_df.sort_values(by='game_name',inplace=True)
topranked_idx = topranked_df.index
demo_gamelist = tuple(list(finalgamelist_df.loc[topranked_idx,'game_name']))

## Pretend some game selected

In [43]:
mygamename = 'magic gathering'
# IF NAME IS UNCLEAR
mygamename,qltynum = get_real_name_fuzzy(mygamename,finalgamelist_df)
print('Best guess: {} (match score: {}/100)'.format(mygamename,str(qltynum)))
mygameid = list(finalgamelist_df.index[finalgamelist_df['game_name']==mygamename])[0] # Need INDEX, not idx
mygameurl=list(allgamedata_df.loc[allgamedata_df['game_name']==mygamename,'bgg_url'])[0]

Best guess: Magic: The Gathering (match score: 88/100)


In [56]:
W1=50
W2=1

In [57]:
mytop10simlist_df = getcompute_similar_games_by_id(mygameid,mygamename,allgamedata_df,allgamedocvects,finalgamelist_df,bgg_gameplay_df, allgamePLAYdocvects,W1,W2)
mygamevect_df = streamlitify_df(mytop10simlist_df)

In [49]:
mygamevect_df

Game  \
1               Pokémon Trading Card Game   
2                               Rabohnzel   
3                       Millennium Blades   
4                  Book of Mormon Battles   
5   My Little Pony: Collectible Card Game   
6             Tales of the Arabian Nights   
7                        Origami: Legends   
8            Buffy the Vampire Slayer CCG   
9           Vampire: The Eternal Struggle   
10                           War of Honor   

                                                  url  \
1   https://boardgamegeek.com//boardgame/2165/poke...   
2   https://boardgamegeek.com//boardgame/20136/rab...   
3   https://boardgamegeek.com//boardgame/151347/mi...   
4   https://boardgamegeek.com//boardgame/27468/boo...   
5   https://boardgamegeek.com//boardgame/149746/my...   
6   https://boardgamegeek.com//boardgame/34119/tal...   
7   https://boardgamegeek.com//boardgame/288431/or...   
8   https://boardgamegeek.com//boardgame/8930/buff...   
9   https://boardgamegeek.com//boardgame/2122/vamp...   
10  https://boardgamegeek.com//boardgame/93819/war...   

                                            Game_link  
1   <a target="_blank" href="https://boardgamegeek...  
2   <a target="_blank" href="https://boardgamegeek...  
3   <a target="_blank" href="https://boardgamegeek...  
4   <a target="_blank" href="https://boardgamegeek...  
5   <a target="_blank" href="https://boardgamegeek...  
6   <a target="_blank" href="https://boardgamegeek...  
7   <a target="_blank" href="https://boardgamegeek...  
8   <a target="_blank" href="https://boardgamegeek...  
9   <a target="_blank" href="https://boardgamegeek...  
10  <a target="_blank" href="https://boardgamegeek...

In [52]:
mygamevect_df

Game  \
1      KeyForge: Age of Ascension   
2   KeyForge: Call of the Archons   
3                          WarCry   
4                          Portal   
5                    WWE Raw Deal   
6        KeyForge: Worlds Collide   
7                          Wyvern   
8     Hatalom Kártyái Kártyajáték   
9     Romance of the Nine Empires   
10                   Four Taverns   

                                                  url  \
1   https://boardgamegeek.com//boardgame/272453/ke...   
2   https://boardgamegeek.com//boardgame/257501/ke...   
3    https://boardgamegeek.com//boardgame/6380/warcry   
4    https://boardgamegeek.com//boardgame/1891/portal   
5   https://boardgamegeek.com//boardgame/18824/wwe...   
6   https://boardgamegeek.com//boardgame/285775/ke...   
7    https://boardgamegeek.com//boardgame/2166/wyvern   
8   https://boardgamegeek.com//boardgame/59650/hat...   
9   https://boardgamegeek.com//boardgame/143701/ro...   
10  https://boardgamegeek.com//boardgame/122867/fo...   

                                            Game_link  
1   <a target="_blank" href="https://boardgamegeek...  
2   <a target="_blank" href="https://boardgamegeek...  
3   <a target="_blank" href="https://boardgamegeek...  
4   <a target="_blank" href="https://boardgamegeek...  
5   <a target="_blank" href="https://boardgamegeek...  
6   <a target="_blank" href="https://boardgamegeek...  
7   <a target="_blank" href="https://boardgamegeek...  
8   <a target="_blank" href="https://boardgamegeek...  
9   <a target="_blank" href="https://boardgamegeek...  
10  <a target="_blank" href="https://boardgamegeek...

In [58]:
mygamevect_df

Game  \
1                 Pokémon Trading Card Game   
2                              War of Honor   
3             Vampire: The Eternal Struggle   
4   The Lord of the Rings Trading Card Game   
5                         Millennium Blades   
6                KeyForge: Age of Ascension   
7                       Munchkin Pathfinder   
8              Buffy the Vampire Slayer CCG   
9                             Summoner Wars   
10                   Nightfall: Martial Law   

                                                  url  \
1   https://boardgamegeek.com//boardgame/2165/poke...   
2   https://boardgamegeek.com//boardgame/93819/war...   
3   https://boardgamegeek.com//boardgame/2122/vamp...   
4   https://boardgamegeek.com//boardgame/2603/lord...   
5   https://boardgamegeek.com//boardgame/151347/mi...   
6   https://boardgamegeek.com//boardgame/272453/ke...   
7   https://boardgamegeek.com//boardgame/129359/mu...   
8   https://boardgamegeek.com//boardgame/8930/buff...   
9   https://boardgamegeek.com//boardgame/58281/sum...   
10  https://boardgamegeek.com//boardgame/96152/nig...   

                                            Game_link  
1   <a target="_blank" href="https://boardgamegeek...  
2   <a target="_blank" href="https://boardgamegeek...  
3   <a target="_blank" href="https://boardgamegeek...  
4   <a target="_blank" href="https://boardgamegeek...  
5   <a target="_blank" href="https://boardgamegeek...  
6   <a target="_blank" href="https://boardgamegeek...  
7   <a target="_blank" href="https://boardgamegeek...  
8   <a target="_blank" href="https://boardgamegeek...  
9   <a target="_blank" href="https://boardgamegeek...  
10  <a target="_blank" href="https://boardgamegeek...

#  ******** ROUGH STUFF ****************